In [34]:
import pandas as pd
import numpy as np
import os
import re
import nltk
from nltk import MWETokenizer, WordNetLemmatizer
from nltk.corpus import wordnet as wn, stopwords
import gensim
from gensim import corpora
from nltk.corpus import reuters

In [42]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocessing(text):
    text = re.sub(r'[^\w\s]',' ',text) # rimuovo la punteggiatura
    text = text.lower()
    text = nltk.pos_tag(text.split()) # prendo i pos tag delle parole (fa anche il tokenizing)
    text = [x for x in text if x[1] in ['NN','NNS','NNP','NNPS']] # mantengo solo i noun
    text = [x[0] for x in text] # rimuovo i pos tag
    text = [lemmatizer.lemmatize(x) for x in text]
    text = [x for x in text if x not in stop_words]
    return text

Il corpus Reuters è un popolare corpus linguistico utilizzato per la ricerca e l'elaborazione del linguaggio naturale. È composto da una vasta collezione di articoli di notizie provenienti dalla Reuters news agency, che copre una vasta gamma di argomenti come politica, economia, finanza, scienza, tecnologia e altro ancora. Il corpus Reuters è spesso utilizzato come benchmark per valutare algoritmi di classificazione del testo, modelli di linguaggio e altre applicazioni nel campo dell'elaborazione del linguaggio naturale. È incluso nella libreria NLTK (Natural Language Toolkit) di Python, che offre un facile accesso a numerosi corpus linguistici.

Contiene 90 argomenti distinti

In [43]:
def get_topics(doc_ids):
    topics = {}
    for doc_id in doc_ids:
        for topic in reuters.categories(doc_id):
            if topic in topics:
                topics[topic] += 1
            else:
                topics[topic] = 1
    return topics

def get_reuters_docs(test=True):
    docs_ids = reuters.fileids()
    if (test):
        docs_ids = [doc_id for doc_id in docs_ids if 'test' in doc_id]

    docs = [preprocessing(reuters.raw(doc_id)) for doc_id in docs_ids]
    topics = get_topics(docs_ids)

    return docs, topics

docs, topics = get_reuters_docs()
print('Numero documenti: ' + str(len(docs)))
print('Numero topic: ' + str(len(topics)))
print('\n')
print(topics)

Numero documenti: 3019
Numero topic: 90


{'trade': 117, 'grain': 149, 'crude': 189, 'nat-gas': 30, 'corn': 56, 'rice': 24, 'rubber': 12, 'sugar': 36, 'tin': 12, 'palm-oil': 10, 'veg-oil': 37, 'ship': 89, 'coffee': 28, 'lumber': 6, 'wheat': 71, 'gold': 30, 'acq': 719, 'interest': 131, 'money-fx': 179, 'copper': 18, 'ipi': 12, 'carcass': 18, 'livestock': 24, 'oilseed': 47, 'soybean': 33, 'earn': 1087, 'bop': 30, 'gas': 17, 'lead': 14, 'jobs': 21, 'zinc': 13, 'cpi': 28, 'gnp': 35, 'soy-oil': 11, 'dlr': 44, 'yen': 14, 'nickel': 1, 'groundnut': 4, 'heat': 5, 'sorghum': 10, 'sunseed': 5, 'pet-chem': 12, 'cocoa': 18, 'rapeseed': 9, 'cotton': 20, 'money-supply': 34, 'iron-steel': 14, 'l-cattle': 2, 'alum': 23, 'palladium': 1, 'platinum': 7, 'strategic-metal': 11, 'reserves': 18, 'groundnut-oil': 1, 'lin-oil': 1, 'meal-feed': 19, 'rape-oil': 3, 'sun-meal': 1, 'sun-oil': 2, 'hog': 6, 'barley': 14, 'potato': 3, 'orange': 11, 'retail': 2, 'soy-meal': 13, 'cotton-oil': 2, 'oat': 6, 'fuel': 10, 'si

In [45]:
"""
I valori dell'output rappresentano i punteggi di coerenza associati ai rispettivi modelli LDA. 
Questi punteggi sono stati ottenuti mediante l'utilizzo del CoherenceModel. Esso calcola la coerenza 
dei topic utilizzando diverse metriche di coerenza, come ad esempio la metrica C_V. 
Queste metriche valutano la somiglianza semantica tra le parole chiave all'interno di ciascun topic, 
considerando anche la distribuzione delle parole nel corpus di testo.
"""
def fine_tuning(texts, limit, start=2, step=2):
    coherence_values = []
    model_list = []
    num_topic_list = []

    dictionary = corpora.Dictionary(texts)
    dictionary.filter_extremes(no_below=5, no_above=0.3, keep_n=None)  # use Dictionary to remove un-relevant tokens
    corpus = [dictionary.doc2bow(doc) for doc in texts]


    for num_topics in range(start, limit, step):
        print("Elaborazione per k = " + str(num_topics))
        model = gensim.models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        num_topic_list.append(num_topics)
        coherencemodel = gensim.models.CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, num_topic_list, coherence_values

In [46]:
model_list, num_topic_list, coherence_values = fine_tuning(docs, 20, start=2, step=3)

Elaborazione per k = 2
Elaborazione per k = 5
Elaborazione per k = 8
Elaborazione per k = 11
Elaborazione per k = 14
Elaborazione per k = 17


In [47]:
for i in range(0, len(num_topic_list)):
    print(str(num_topic_list[i]) + " topic : " + str(coherence_values[i]))

2 topic : 0.2799521577712141
5 topic : 0.31912280688987316
8 topic : 0.4115720817156091
11 topic : 0.3993918954172529
14 topic : 0.3893383772284575
17 topic : 0.38123543247628744


In [48]:
for i in range(0, len(num_topic_list)):
    print(f"Modello con {num_topic_list[i]} topic: \n")
    for topic_id, topic in model_list[i].show_topics(formatted=True, num_topics=num_topic_list[i], num_words=10):
        print(f"Topic {topic_id}: {topic}")
    print('\n')

Modello con 2 topic: 

Topic 0: 0.024*"pct" + 0.021*"company" + 0.021*"share" + 0.012*"market" + 0.012*"bank" + 0.012*"stock" + 0.011*"group" + 0.010*"year" + 0.009*"price" + 0.008*"trade"
Topic 1: 0.098*"v" + 0.025*"loss" + 0.023*"shr" + 0.020*"mths" + 0.018*"year" + 0.016*"profit" + 0.014*"oil" + 0.014*"rev" + 0.013*"sale" + 0.011*"avg"


Modello con 5 topic: 

Topic 0: 0.042*"pct" + 0.025*"year" + 0.022*"tonne" + 0.018*"price" + 0.017*"oil" + 0.016*"rate" + 0.010*"company" + 0.009*"market" + 0.007*"crop" + 0.007*"month"
Topic 1: 0.063*"v" + 0.036*"share" + 0.020*"bank" + 0.017*"sale" + 0.015*"pct" + 0.015*"shr" + 0.015*"company" + 0.014*"corp" + 0.014*"stock" + 0.014*"loan"
Topic 2: 0.019*"oil" + 0.017*"pct" + 0.014*"trade" + 0.014*"market" + 0.014*"year" + 0.012*"price" + 0.012*"rate" + 0.011*"export" + 0.010*"bank" + 0.009*"state"
Topic 3: 0.108*"v" + 0.038*"loss" + 0.028*"share" + 0.025*"shr" + 0.024*"profit" + 0.024*"mths" + 0.019*"company" + 0.019*"rev" + 0.015*"stock" + 0.015*